# Datascraping : Web Scraping / APIs
Dec 10th, 2018 - Javier Garcia-Bernardo, Anna Keuchenius & Allie Morgan

In [1]:
## Requirements
import requests               # Simple HTTP operations (GET and POST)
import selenium               # Loads dynamic (javascript) pages
import json                   # Parsing the responses from APIs
import re                     # Python library for parsing regular expressions
from bs4 import BeautifulSoup # Parsing HTML
import pandas as pd           # Read tables
import time

## What is datascraping?

[Data scraping](https://en.wikipedia.org/wiki/Web_scraping) is a method for extracting data from the web. There are many techniques which can be used for web scraping — ranging from requiring human involvement (“human copy-paste”) to fully automated systems (using computer vision). Somewhere in the middle is HTML parsing, which we will describe here.

Web scraping using [HTML parsing](https://en.wikipedia.org/wiki/Web_scraping#HTML_parsing) is often used on webpages which share similar HTML structure. For example, you might want to scrape the ingredients from chocolate chip cookie recipes to identify correlations between ingredients and five-star worthy cookies, or you might want to predict who will win March Madness by looking at game play-by-plays, or you want to know all the local pets up for adoption.

## Part I: Static Webpages

### Example of basics request

#### Example 1

In [18]:
r = requests.get('https://cssamsterdam.github.io/')

In [19]:
r.status_code

200

In [20]:
r.ok

True

In [21]:
r.text[0:1000]

u'<!DOCTYPE html>\r\n<!--[if IE 8 ]><html class="no-js oldie ie8" lang="en"> <![endif]-->\r\n<!--[if IE 9 ]><html class="no-js oldie ie9" lang="en"> <![endif]-->\r\n<!--[if (gte IE 9)|!(IE)]><!--><html class="no-js" lang="en"> <!--<![endif]-->\r\n<head>\r\n\r\n   <!--- basic page needs\r\n   ================================================== -->\r\n   <meta charset="utf-8">\r\n\t<title>Computational Social Science -- Amsterdam</title>\r\n\t<meta name="description" content="">  \r\n\t<meta name="author" content="">\r\n\r\n   <!-- mobile specific metas\r\n   ================================================== -->\r\n\t<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">\r\n\r\n \t<!-- CSS\r\n   ================================================== -->\r\n   <link rel="stylesheet" href="css/base.css">  \r\n   <link rel="stylesheet" href="css/main.css">\r\n   <link rel="stylesheet" href="css/vendor.css">  \r\n\r\n\r\n   <!-- script\r\n   =======================

#### Example 2

In [46]:
urls = ["https://www.boulderhumane.org/animals/adoption/dogs", 
         "https://www.boulderhumane.org/animals/adoption/cats", 
         "https://www.boulderhumane.org/animals/adoption/adopt_other"]

page = requests.get(urls[0])
# Extractt 
html = page.text
print(html[:500]) # Print the first 500 characters of the HTML

<!DOCTYPE html>
<head>
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta charset="utf-8" />
<link rel="shortcut icon" href="https://www.boulderhumane.org/sites/default/files/favicon.ico" type="image/vnd.microsoft.icon" />
<meta name="Generator" content="Drupal 7 (http://drupal.org)" />
<meta name="viewport" content="width=1000px, initial-scale=1.0, maximum-scale=1.0" />
<title>Dogs Available for Adoption | Humane Society of Boulder Valley</title>
<link type="text/css" rel="stylesheet


### Extracting information: parsing html

When you visit a webpage, your web browser renders an HTML document with CSS and Javascript to produce a visually appealing page. (See the HTML above.) [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) is a Python library for parsing HTML. We'll use it to extract all of the names, ages, and breeds of the [dogs](https://www.boulderhumane.org/animals/adoption/dogs), [cats](https://www.boulderhumane.org/animals/adoption/cats), and [small animals](https://www.boulderhumane.org/animals/adoption/adopt_other) currently up for adoption at the Boulder Humane Society.

In [6]:
soup = BeautifulSoup(html, 'html.parser')

Note, that the feature of these pages which we are exploiting is their repeated HTML structure. Every animal listed has the following HTML variant:
```{html}
<div class="views-row ... ">
  ...
  <div class="views-field views-field-field-pp-animalname">
    <div class="field-content">
      <a href="/animals/adoption/" title="Adopt Me!">Romeo</a>
    </div>
  </div>
  <div class="views-field views-field-field-pp-primarybreed">
    <div class="field-content">New Zealand</div>
  </div>
  <div class="views-field views-field-field-pp-secondarybreed">
    <div class="field-content">Rabbit</div>
  </div>
  <div class="views-field views-field-field-pp-age">
    ...
    <span class="field-content">0 years 2 months</span>
  </div>
  <div class="views-field views-field-field-pp-gender">
    ...
    <span class="field-content">Male</span>
  </div>
  ...
</div>
``` 
So to get at the HTML object for each pet, we can run the following:

In [13]:
pets = soup.find_all('div', {'class': re.compile('.*views-row.*')})

That is, find all of the `div` tags with the `class` attribute which contains the substring `views-row`. 

remark: notice that we use regex here in the re.compile statement. Regex helps to find patterns in text. More info on regex [here](https://docs.python.org/3/library/re.html). The wildcard .* matches everything (all characters).

Next, to grab the name, breeds, and ages of these pets, we’ll grab the children of each pet HTML object. For example:

In [17]:
head = "views-field views-field-field-pp-"
for pet in pets:
    name = pet.find('div', {'class': head + 'animalname'}).get_text(strip=True)
    primary_breed = pet.find('div', {'class': head + 'primarybreed'}).get_text(strip=True)
    secondary_breed = pet.find('div', {'class': head + 'secondarybreed'}).get_text(strip=True)
    age = pet.find('div', {'class': head + 'age'}).get_text(strip=True)
    print(name, primary_breed, secondary_breed, age)

(u'Blue', u'Coonhound, Treeing Walker', u'', u'Age:7 years 5 months')
(u'Calvin', u'Pointer, German Shorthaired', u'Mix', u'Age:1 year 9 months')
(u'Mona', u'Greyhound', u'Retriever, Labrador', u'Age:8 years 1 month')
(u'Iris', u'Siberian Husky', u'Saint Bernard', u'Age:7 years 0 months')
(u'Lucia', u'Chihuahua, Short Coat', u'Mix', u'Age:6 years 0 months')
(u'Karma', u'Terrier, American Pit Bull', u'Mix', u'Age:3 years 0 months')
(u'Chuco', u'Terrier, American Pit Bull', u'Mix', u'Age:3 years 7 months')
(u'Harper', u'Terrier, American Pit Bull', u'Mix', u'Age:1 year 1 month')
(u'Lupita', u'Bulldog, American', u'Mix', u'Age:2 years 0 months')
(u'Winston', u'Chihuahua, Short Coat', u'Mix', u'Age:1 year 0 months')
(u'Beans', u'Terrier, American Pit Bull', u'Mix', u'Age:0 years 2 months')
(u'Coco', u'Bulldog, English', u'Mix', u'Age:0 years 5 months')
(u'Oliver', u'Alaskan Klee Kai', u'', u'Age:7 years 10 months')
(u'Canon', u'Siberian Husky', u'Mix', u'Age:3 years 0 months')
(u'Eve', u'M

where each call to `find` is getting the children of a pet object, in particular, the `div`s with `class` attributes which look like `views-field views-field-field-pp-*`. Feel free to replace the above code with the cat or small animal pages provided and see how the output changes.

We often use BeautifulSoup's .find or .find_all methods. However, there are also other useful methods that one can use.  For example, .findNext of .nextSibling. More in the [docs](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) (especially under tab 'method names'). 

### Part I.3: Extract Tables from Webpages

We can quickly read a table from html and converge it to a Pandas dataframe with pandas method read_html. Then we can use the regular pandas methods to manipulate or filter the data.

In [19]:
table = pd.read_html("https://en.wikipedia.org/wiki/List_of_sandwiches",header=0)[0]

# Write table to CSV
#table.to_csv("filenamehere.csv")

# Output the top rows of the table
table.head((20))

,Name,Image,Origin,Description
0,Bacon,NaN,United Kingdom,Often eaten with ketchup or brown sauce
1,"Bacon, egg and cheese",NaN,United States,"Breakfast sandwich, usually with fried or scra..."
2,Bagel toast,NaN,Israel,"Pressed, toasted bagel filled with vegetables ..."
3,Baked bean,NaN,United States,"Canned baked beans on white or brown bread, so..."
4,Bánh mì[4],NaN,Vietnam,"Filling is typically meat, but can contain a w..."
5,Barbecue[5][6][7],NaN,United States,"Served on a bun, with chopped, sliced, or shre..."
6,Barros Jarpa,NaN,Chile,"Ham and cheese, usually mantecoso, which is si..."
7,Barros Luco,NaN,Chile,Beef (usually thin-cut steak) and cheese
8,Bauru,NaN,Brazil,"Melted cheese, roast beef, tomato, and pickled..."
9,Beef on weck,NaN,"United States(Buffalo, New York)",Roast beef on a Kummelweck roll


Then we can use the regular pandas manipulation methods to filter this dataframe or to make changes. For example, we can filter for only animals from the UK as follows:

In [35]:
animals_from_uk = table[table['Origin'] == 'United Kingdom']
animals_from_uk.head()

,Name,Image,Origin,Description
0,Bacon,NaN,United Kingdom,Often eaten with ketchup or brown sauce
19,British Rail,NaN,United Kingdom,Reference to the poor quality of catering on t...
29,Cheese and pickle,NaN,United Kingdom,Slices of cheese (typically Cheddar) and pickl...
37,Chip butty[11][12][13][14],NaN,United Kingdom,"Sliced white bread (or a large, flat bread rol..."
44,Corned beef,NaN,United Kingdom,Corned beef often served with a condiment such...


## Part  II: Dynamic Webpages

Above, we requested webpages that required no [Javascript](https://en.wikipedia.org/wiki/JavaScript). In other words, there was no input required on the users' end to view the content of the page (e.g. a login). Let's try a more complicated example of webscraping where content is loaded dynamically.

[Selenium](https://www.seleniumhq.org/download/)

Some advantages of HTML scraping with Selenium it: can handle javascript, get **HTML** back after the Javascript has been rendered, can behave like a person. The disadvantage of using Selenium is that it is generally slow.

Requirements (one of the below):
- Firefox + geckodriver (https://github.com/mozilla/geckodriver/releases)
- Chrome + chromedriver (https://sites.google.com/a/chromium.org/chromedriver/)
    
Note: geckodriver/chromedriver must have execution permissions (chmod +x geckodriver)

In [32]:
import selenium.webdriver

In [38]:
# Open the driver (change the executable path to geckodriver_mac.exe or geckodriver.exe)
# driver = selenium.webdriver.Chrome(executable_path="./chromedriver")
driver = selenium.webdriver.Chrome()

Visit [xkcd](https://xkcd.com) and click through the comics.

In [43]:
# Get the xkcd website
driver.get("https://xkcd.com/")

In [37]:
# Let's find the 'random' buttom and go to a random comic
# * Yesterday this gave a time-out error, meaning something went wrong in the webserver
# If this happens again, rerun last cell and skip this cell 
random_element = driver.find_element_by_xpath('//*[@id="middleContainer"]/ul[1]/li[3]/a')
random_element.click()

In [50]:
# Identify the 'previous' button and go to the next comic
next_element = driver.find_element_by_css_selector("a[rel='prev']")
next_element.click()

As you can see in the examples above, there are several ways to identify elements on the page, such as with the xpath or by css selectors. You can find more methods in the [docs](https://onlinetraining.etestinghub.com/webdriver-methods-web-elements/).

Find an attribute of this page.

In [58]:
# Get the title of the comic
element_title = driver.find_element_by_id("ctitle")
element_title.text

u'Alpha Centauri'

In [59]:
# Get the title of the comic image (not visible on the page in the browser!)
element = driver.find_element_by_xpath('//*[@id="comic"]/img')
element.get_attribute("title")

u"And let's be honest, it's more like two and a half stars. Proxima is barely a star and barely bound to the system."

Notice how the url changes as you click through the comics. If we would want to download all the comics from this website, we don't necessary need to click through via selenium. Instead, we could simply get (via requests) all these urls with changes integer "https://xkcd.com/<integer>". This would be much faster.  

### Login with Selenium

Let's visit a webpage which requires a login. Signing in to Facebook ...

In [62]:
##DO NOT WRITE YOUR PASSWORD IN NOTEBOOKS!!
# Write your email below
fb_email = "annakeuchenius@hotmail.com"

In [63]:
# Go to Facebook
driver = selenium.webdriver.Chrome(executable_path="./chromedriver")
driver.get("https://www.facebook.com/")

In [66]:
# Send email and password
driver.find_element_by_xpath('//*[@id="email"]').send_keys(fb_email)
print("Now fill out your password, don't hit enter and move on to the next cell in this notebook")
# driver.find_element_by_xpath('//*[@id="pass"]').send_keys(fb_pass)

Now fill out your password, don't hit enter and move on to the next cell in this notebook


In [67]:
# Click on login
driver.find_element_by_xpath('//*[@id="loginbutton"]').click()

In [72]:
# Go to CSS Amsterdam Website
driver.get("https://www.facebook.com/CSSamsterdam/")

In [73]:
vind_ik_leuks = driver.find_element_by_xpath("//*[contains(text(), 'Vind ik leuk')]").click()

In [74]:
# Always remember to close your browser!
driver.close()

Notice that we have now required you fill out your password manually. However, usually when scraping you don't want to do this manually, but instead you want to automate it. Of course you could put your password in your  notebook and fill them out on the website automatically, just like with your email. However, from a security perspective, this is not your best solution. Instead, you can put your passwords in a file, and make your notebook read the password from the file. It is especially important not to put your passwords in your notebooks when you share your scripts directly with others, or submit them to a gitserver (like github).

## Part III: APIs

To allow users to access large amounts of data, companies may provide an [Application Programming Interface (API)](https://en.wikipedia.org/wiki/Application_programming_interface). Often these request are handled via PUT and POST HTTP requests. For example, to make a request from the Twitter API:

```{bash}
curl --request GET 
 --url 'https://api.twitter.com/1.1/search/tweets.json?q=nasa&result_type=popular' 
 --header 'authorization: OAuth oauth_consumer_key="consumer-key-for-app", ... , 
 oauth_token="access-token-for-authed-user", oauth_version="1.0"'
 ```

APIs often return data in the format of [Javascript Object Notation (JSON)](https://en.wikipedia.org/wiki/JSON). For example:

```{json}
{"status": 200, "message": "hello world"}
```

### Explicit APIs

Next, let's try a more typical example of an API. The perks of this approach: 
- (a) send a request and get back JSON, 
- (b) chances are that somebody else has created a Python wrapper for you, but keep in mind that 
- (c) APIs have limits.

Let's consider a common API example -- Twitter. To get started:
- Get a key: https://apps.twitter.com/
- Documentation: https://dev.twitter.com/rest/public
- Find a library: https://dev.twitter.com/resources/twitter-libraries (We'll use https://github.com/tweepy/tweepy)

Limitations: 100 messages / query, 180 messages every 15 min, & only the last seven days of data 

In [104]:
!pip install tweepy

Below you will find the twitter keys that were created for this demo. Please don't use these keys after this datascraping course any longer. GENEREALLY, NEVER STORE PASSWORD, KEYS OR OTHER SENSITIVE INFORMATION IN NOTEBOOKS. * We will also regenerate these keys after today, so that the ones below are no longer valid. 

In [84]:
d_keys = {}
d_keys['CONSUMER_KEY'] = 'iISHnFwAbqlBehAP6RxUIjawG'
d_keys['CONSUMER_SECRET'] = 'XJTe1LlhZMSaRHE7SH1paZGwzb2HNbd5GyeDgI9HoLVoeBoBVM'
d_keys['ACCESS_KEY'] = '3001441779-A3BEX2aY86j7yOsjwiZ7bRYmFkHnfpsSaPVtaBs'
d_keys['ACCESS_SECRET'] = 'qa1ZKpMZPmKqoKJUvD4Gdw3GWhuMRHtIrFvkR7DYoaLHc'

In [92]:
import tweepy
import time
import pickle

def twitter(d_keys,query, num_results=1000):
    # Authtentify
    auth = tweepy.OAuthHandler(d_keys["CONSUMER_KEY"], d_keys["CONSUMER_SECRET"])
    auth.set_access_token(d_keys["ACCESS_KEY"], d_keys["ACCESS_SECRET"])
    api = tweepy.API(auth)

    # We want 1000 tweets
    result_count = 0
    last_id = None
    
    # Max 180 tweets 15 min
    cumulative = 1

    #While we don't have them
    while (result_count <  num_results):
        previous_tweets = result_count
        # Ask for more tweets, starting in the 'last_id' (identifier of the tweet)
        results = api.search(q = query,
                              count = 90, max_id = last_id, result_type="recent")
                                # geocode = "{},{},{}km".format(latitude, longitude, max_range) #for geocode

        # For each tweet extract some info (JSON structure)
        for result in results:
            result_count += 1
            user = result.user.screen_name
            text = result.text
            followers_count = result.user.followers_count
            time_zone = result.user.time_zone
            print("_"*10)
            print(user,time_zone,followers_count)
            print(text)

        # Keep the last_id to know where to continue
        last_id = int(result.id)-1
        new_tweets = result_count - previous_tweets

        print ("Number of results: {} ({} new)".format(result_count,new_tweets))

        # If we don't get new tweets exit
        if new_tweets == 0: 
            break
        
        time.sleep(1)
        
        if ((result_count + 90) // 150) > cumulative:
            cumulative += 1
            time.sleep(15*60)


twitter(d_keys,"ic2s2", num_results=10)

__________
(u'ODISSEI_nl', None, 102)
ODISSEI proudly cooperates with the @IC2S2 conference on Computational Social Science in Amsterdam. https://t.co/chyjPIhJ9A
__________
(u'MNoel75', None, 446)
RT @IC2S2: Call for papers for #IC2S2 in Amsterdam is now online: https://t.co/Z6nJB7kA8n The deadline for tutorial proposals is January 4,…
__________
(u'nicbaya', None, 879)
RT @IC2S2: Call for papers for #IC2S2 in Amsterdam is now online: https://t.co/Z6nJB7kA8n The deadline for tutorial proposals is January 4,…
__________
(u'jpgcrowley', None, 474)
RT @IC2S2: Call for papers for #IC2S2 in Amsterdam is now online: https://t.co/Z6nJB7kA8n The deadline for tutorial proposals is January 4,…
__________
(u'EASSH_', None, 235)
RT @IC2S2: Call for papers for #IC2S2 in Amsterdam is now online: https://t.co/Z6nJB7kA8n The deadline for tutorial proposals is January 4,…
__________
(u'Akbaritabar', None, 1492)
RT @IC2S2: Call for papers for #IC2S2 in Amsterdam is now online: https://t.co/Z6nJB7kA8n Th

# Advanced Scraping Issues
please see note book css2018-webscraping-advanced